# Summary

noteobook to train the model 4

## Extract name entites from CoNLL-2003

In [ ]:
!git clone https://github.com/practical-nlp/practical-nlp-code

Cloning into 'practical-nlp-code'...
remote: Enumerating objects: 1635, done.
remote: Counting objects: 100% (348/348), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 1635 (delta 199), reused 273 (delta 145), pack-reused 1287
Receiving objects: 100% (1635/1635), 90.66 MiB | 19.35 MiB/s, done.
Resolving deltas: 100% (919/919), done.
Checking out files: 100% (159/159), done.


In [ ]:
!python -m spacy convert "train.txt" spacyNER_data -c ner
!python -m spacy convert "test.txt" spacyNER_data -c ner
!python -m spacy convert "valid.txt" spacyNER_data -c ner

In [ ]:
import spacy
from spacy.tokens import DocBin

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc_bin = DocBin().from_disk("/content/conll2003/train.spacy")

In [ ]:
docs = list(doc_bin.get_docs(nlp.vocab))

In [ ]:
ents = [n.ents for n in docs if len(n.ents)>0 ]

## Load dataset and attributes from the sample Cicero dataset

In [ ]:
import re
import pandas as pd
from random import randrange

In [ ]:
sample = pd.read_csv('/content/drive/MyDrive/Cicero/cicero_officials_sample_2022-09-08.csv', \
                       error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 13: expected 55 fields, saw 60\nSkipping line 34: expected 55 fields, saw 59\nSkipping line 110: expected 55 fields, saw 60\nSkipping line 125: expected 55 fields, saw 60\nSkipping line 127: expected 55 fields, saw 60\nSkipping line 171: expected 55 fields, saw 60\nSkipping line 175: expected 55 fields, saw 65\nSkipping line 181: expected 55 fields, saw 60\nSkipping line 196: expected 55 fields, saw 60\nSkipping line 197: expected 55 fields, saw 58\nSkipping line 206: expected 55 fields, saw 60\nSkipping line 220: expected 55 fields, saw 60\nSkipping line 261: expected 55 fields, saw 60\nSkipping line 272: expected 55 fields, saw 63\nSkipping line 279: expected 55 fields, saw 60\nSkipping line 353: expected 55 fields, saw 58\nSkipping li

In [ ]:
sample.columns

Index(['id', 'party', 'initial_term_start_date',
       'initial_term_start_date_precision', 'valid_from',
       'valid_from_precision', 'valid_to', 'valid_to_precision', 'first_name',
       'middle_initial', 'last_name', 'name_suffix', 'primary_address_1',
       'primary_address_2', 'primary_address_3', 'primary_city',
       'primary_county', 'primary_state', 'primary_postal_code',
       'primary_phone_1', 'primary_phone_2', 'primary_fax_1', 'primary_fax_2',
       'secondary_address_1', 'secondary_address_2', 'secondary_address_3',
       'secondary_city', 'secondary_county', 'secondary_state',
       'secondary_postal_code', 'secondary_phone_1', 'secondary_phone_2',
       'secondary_fax_1', 'secondary_fax_2', 'email_1', 'email_2',
       'photo_origin_url', 'description_1', 'description_2', 'notes', 'url_1',
       'url_2', 'salutation', 'web_form_url', 'role', 'trans_from',
       'last_update_user', 'trans_to', 'is_staging', 'sk', 'chamber_id',
       'district_id', 'version

In [ ]:
address2 = pd.read_csv('/content/drive/MyDrive/Cicero/cicero_officials_sample_2022-09-08.csv', \
                       error_bad_lines=False)[['last_name', 'first_name', 'primary_phone_1',\
                                               'email_1',\
                                               'primary_address_1', 'primary_address_2', 'primary_address_3',\
                                               'primary_city','primary_state', 'primary_postal_code']]

b'Skipping line 13: expected 55 fields, saw 60\nSkipping line 34: expected 55 fields, saw 59\nSkipping line 110: expected 55 fields, saw 60\nSkipping line 125: expected 55 fields, saw 60\nSkipping line 127: expected 55 fields, saw 60\nSkipping line 171: expected 55 fields, saw 60\nSkipping line 175: expected 55 fields, saw 65\nSkipping line 181: expected 55 fields, saw 60\nSkipping line 196: expected 55 fields, saw 60\nSkipping line 197: expected 55 fields, saw 58\nSkipping line 206: expected 55 fields, saw 60\nSkipping line 220: expected 55 fields, saw 60\nSkipping line 261: expected 55 fields, saw 60\nSkipping line 272: expected 55 fields, saw 63\nSkipping line 279: expected 55 fields, saw 60\nSkipping line 353: expected 55 fields, saw 58\nSkipping line 356: expected 55 fields, saw 59\nSkipping line 407: expected 55 fields, saw 57\nSkipping line 429: expected 55 fields, saw 60\nSkipping line 446: expected 55 fields, saw 69\nSkipping line 507: expected 55 fields, saw 60\nSkipping line

## Load the essay dataset that is used as the body content to be inserted

In [ ]:
essay = pd.read_table('/content/drive/MyDrive/spaCy Address Model/test_set.tsv', encoding='mac_roman')

## Insert

Attributes including names, phone numbers, emails, cities, states, and post codes are inserted but not tagged in the customized dataset to increase the difficulty of finding correct address information in the content.

In [ ]:
import collections
from spacy.util import filter_spans
from spacy.tokens import Doc, DocBin
import random
from tqdm import tqdm

In [ ]:
interted_data = []
for n in tqdm(range(len(address2))):
  address_list = list(address2.iloc[n].dropna())
  random_ents = [list(n) for n in random.choices(ents, k=4)]
  random_ents = [str(n) for n in sum(random_ents, [])]
  address_list = address_list + random_ents
  raw_essay = essay.iloc[n]['essay']
  ## insert
  essay_sentences = [str(s) for s in nlp(raw_essay).sents]

  for m in range(len(address_list)):
    random_index = randrange(len(essay_sentences))
    essay_sentences = essay_sentences[:random_index] + [address_list[m]] + essay_sentences[random_index:]
  ## BIO tagging
  attribute_dict = dict(address2.iloc[n].dropna())
  ruler = nlp.add_pipe("span_ruler")
  patterns = []
  ## address
  if 'primary_address_1' in attribute_dict:
    patterns.append({"label":'ADDRESS', 'pattern':attribute_dict['primary_address_1']})
  if 'primary_address_2' in attribute_dict:
    patterns.append({"label":'ADDRESS', 'pattern':attribute_dict['primary_address_2']})
  if 'primary_address_3' in attribute_dict:
    patterns.append({"label":'ADDRESS', 'pattern':attribute_dict['primary_address_3']})
  # ## pcode
  # if 'primary_city' in attribute_dict:
  #   patterns.append({"label":'CITY', 'pattern':attribute_dict['primary_city']})
  # ## state
  # if 'primary_state' in attribute_dict:
  #   patterns.append({"label":'STATE', 'pattern':attribute_dict['primary_state']})
  # ## post code 
  # if 'primary_postal_code' in attribute_dict:
  #   patterns.append({"label":'STATE', 'pattern':attribute_dict['primary_postal_code']})
  ruler.add_patterns(patterns)
  text = ' '.join(essay_sentences)
  doc = nlp(text)
  doc.ents = filter_spans(doc.spans["ruler"])
  dataset = []
  dataset.append(doc)
  interted_data.append(doc)
  nlp.remove_pipe("span_ruler")

  4%|▍         | 79/1919 [00:13<05:38,  5.44it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
  4%|▍         | 81/1919 [00:14<05:06,  6.00it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
  4%|▍         | 84/1919 [00:14<05:16,  5.80it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
  5%|▌         | 105/1919 [00:18<05:28,  5.52it/s]/usr/local/lib/python3.7/dist-packages/spacy/pipeline/span_ruler.py:507: UserWarning: [W036] The component 'span_ruler' does not have any patterns defined.
  warnings.warn(W

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, dev = train_test_split(interted_data, test_size=0.1, random_state=42)

In [ ]:
train_db = DocBin()
for n in train:
  train_db.add(n)

In [ ]:
train_db.to_disk('/content/train.spacy')

In [ ]:
dev_db = DocBin()
for n in dev:
  dev_db.add(n)

In [ ]:
dev_db.to_disk('/content/dev.spacy')

## Train the model

In [ ]:
!python -m spacy download en_core_web_lg

2022-10-10 01:28:46.663199: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 9.2 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
cp  '/content/drive/MyDrive/worddata_10 08 2022/base_config.cfg' /content/base_config.cfg

In [ ]:
!python -m spacy init fill-config 'base_config.cfg' config.cfg

2022-10-10 01:31:52.469770: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train /content/train.spacy --paths.dev /content/dev.spacy

2022-10-10 01:33:02.151585: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-10 01:33:02,925] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-10 01:33:02,937] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-10-10 01:33:02,942] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-10 01:33:04,920] [INFO] Added vectors: en_core_web_lg
INFO:spacy:Added vectors: en_core_web_lg
tcmalloc: large alloc 1233977344 bytes == 0x3889e000 @  0x7ff84a9052a4 0x7ff83f1fca7d 0x7ff83f1fbc5d 0x7ff83f1f8435 0x7ff83f1f8bde 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51837f 0x7ff76051e218 0x7ff7605201de 0x7ff7605270d7 0x7ff76052a7df 0x590055 0x51b180 0x58f2a7 0x51740e 0x7ff76051e218 0x7ff7605201de 0x7ff760526db0 0x58ed

## Check the results

In [ ]:
from spacy import displacy

In [ ]:
trained_nlp = spacy.load("/content/practical-nlp-code/Ch5/Data/conll2003/en/model-best")

In [ ]:
texts = "Lt. Governor Garlin Gilchrist II Skip to main content Whitmer Governor Gretchen Whitmer About About collapsed link News News collapsed link Issues & Accomplishments Issues & Accomplishments collapsed link Appointments Lt Governor Lt Governor collapsed link Contact the Governor About About the Governor About the Lt Governor About the Governor Governor Gretchen Whitmer is a lifelong Michigander.  She is a lawyer, an educator, former prosecutor, State Representative and Senator.  She was the first woman to lead a Senate caucus. But the most important title she boasts is MOM. Inspired by her family, she’s devoted her life to building a stronger Michigan for everyone. About the Lt Governor News Press Releases Budget State Orders and Directives Flag Honors State of the State Proclamations Press Releases Press Releases Budget Budget State Orders and Directives State Orders and Directives Flag Honors State of the State 2022 State of the State 2021 State of the State 2020 State of the State 2019 State of the State Proclamations Proclamations Issues & Accomplishments Accomplishments Michigan Infrastructure Office Sunshine Tribal Affairs Accomplishments Accomplishments Michigan Infrastructure Office Sunshine Sunshine Tribal Affairs Appointments Lt Governor Thriving Cities Thriving Cities Thriving Cities Tour Ann Arbor Battle Creek Bay City Benton Harbor Dearborn Detroit Flint Grand Rapids Inkster Jackson Kalamazoo Lansing Marquette Muskegon Pontiac Saginaw Southfield Warren Ypsilanti Contact the Governor collapsed link Governor Gretchen Whitmer About News Issues & Accomplishments Appointments Lt Governor Contact the Governor Back About About the Governor About the Lt Governor Back About the Governor Back About the Lt Governor Back News Press Releases Budget State Orders and Directives Flag Honors State of the State Proclamations Back Press Releases Back Budget Back State Orders and Directives Back Flag Honors Back State of the State 2022 State of the State 2021 State of the State 2020 State of the State 2019 State of the State Back Proclamations Back Issues & Accomplishments Accomplishments Michigan Infrastructure Office Sunshine Tribal Affairs Back Accomplishments Back Michigan Infrastructure Office Back Sunshine Back Tribal Affairs Back Appointments Back Lt Governor Thriving Cities Back Thriving Cities Ann Arbor Battle Creek Bay City Benton Harbor Dearborn Detroit Flint Grand Rapids Inkster Jackson Kalamazoo Lansing Marquette Muskegon Pontiac Saginaw Southfield Warren Ypsilanti Back Contact the Governor Header Search Box Label {D47FBCAD-4189-4E1B-B94C-362637BF389B} https://dev.michigan.local/som/json?sc_device=json WHITMER Popular on michigan.gov Agriculture and Rural Development Civil Rights Environment Health and Human Services Natural Resources Secretary of State How Do I... Register to Vote Renew My License Plate View assistance programs Unsupported Browser Detected The web Browser you are currently using is unsupported, and some features of this site may not work as intended. Please update to a modern browser such as Chrome, Firefox or Edge to experience all features Michigan.gov has to offer. Supported Browsers Google Chrome Safari Microsoft Edge Firefox Lt. Governor Garlin Gilchrist II Photo of the Lt Governor with his family. Lt. Governor Garlin Gilchrist II Whitmer Lt Governor Lt Governor Gilchrist High Resolution Headshot Lt. Governor Garlin Gilchrist II Thriving Cities Share Your Opinion Schedule Requests Download Official High-Resolution Headshot of Lt. Governor Gilchrist Lieutenant Governor Garlin Gilchrist has dedicated his career to solving problems. An engineer by training, he uses thoughtful innovation, progressive reform, and efficient modernization of policies and programs to make the work better for hardworking families. From spearheading campaigns for equality and justice to harnessing technology to solve real problems, his focus has consistently remained on serving the public and getting things done. He brings a lifetime of experience to the task, receiving his first computer at age five and setting up a computer lab in a Detroit community recreation center using computers that he built himself at age 16. Lt. Governor Gilchrist studied computer engineering and computer science at the University of Michigan, graduating with honors, and later had a successful career as a software engineer at Microsoft, helping to build SharePoint into the fastest growing product in the company’s history. Lt. Governor Gilchrist spent the next several years of his career as a social entrepreneur and community organizer who helped people in communities across America realize their full economic and political potential. Upon returning home to Detroit, Lt. Governor Gilchrist worked in local government to make sure build systems that made the government more responsive, transparent, and accountable to city residents. This experience prepared him for a key priority: to bring Michigan’s state government fully into the 21 st century on every level. As part of the Whitmer Administration, Lt. Governor Gilchrist has sought to address injustice and inequity across our state at every level. From co-chairing the Michigan Joint Task Force on Jail and Pretrial Incarceration to helming the Coronavirus Task Force on Racial Disparities, these task forces have taken actions that saved lives and protect our most vulnerable populations. Lt. Governor Gilchrist’s approach to addressing these disparities is deeply rooted in fact-based practices, science, and connecting with individuals across Michigan. Lt. Governor Garlin Gilchrist and his wife, Ellen, reside in Detroit where they are raising their twins, Emily and Garlin III, and daughter, Ruby. Social Media Powered by Curator.io Follow us Lt. Governor Garlin Gilchrist II Copyright  State of Michigan Careers Balancing a rewarding career with the ability to have a high-quality lifestyle is important. In Michigan, you can have both. If you’re interested in a career that allows you to make a difference, check out the state of Michigan career portal for a list of current openings in state government. Careers Contact the Governor Governor Whitmer’s office is available to assist Michiganders with matters pertaining to a state government department and agencies. To share an opinion with the Governor’s office or to request assistance with matters pertaining to state government, please follow the link below. Contact the Governor Sign-up for Email Alerts Submit Copyright 2022 State of Michigan Mi.gov Home Policies Accessibility Disability Resources Statewide FOIA Directory Departments"

In [ ]:
doc = trained_nlp(texts)
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
texts ="Jeff M. Witte Director/Secretary of Agriculture Skip to main content New Mexico Department of Agriculture Search our site: Assistance Programs Forms & Applications Licensing & Registration Inspections Statutes & Rules Divisions Divisions Public Records Request Agricultural & Environmental Services: Agricultural & Environmental Services Entomology & Nursery Industries NMDA Hemp Program Pesticide Compliance Agricultural Production Services: Biosecurity Dairy: General Information Feed, Seed, and Fertilizer: Commercial Feed: General Information Fertilizer and Soil Conditioners Seed: General Information FSMA - Produce Safety: Food Safety Modernization Act Food Safety Resources Agricultural Programs & Resources: Acequia and Community Ditch Fund Agricultural Workforce Development Program Healthy Soil Program Noxious Weed Information Rangeland and Grazing Issues SWCC Agendas and Approved Minutes Water and Natural Resource Policy Soil and Water Conservation Districts: Districts Open Act Meetings Resources References and Resources for SWCD Director's/Secretary's Office: Director's/Secretary's Office Laboratories: Petroleum Standards Laboratory Metrology Laboratory State Chemist Laboratory State Seed Laboratory Veterinary Diagnostic Services Marketing & Development: Competitive Grant Programs International Marketing New Mexico-Taste The Tradition® NMDA Organic Program Standards & Consumer Services: Standards & Consumer Services Veterinary Diagnostic Services: Veterinary Diagnostic Services Apiaries Commercial Feed Inspections Dairy Inspections Egg Inspections Fertilizer and Soil Conditioners Fruit and Vegetable Inspections Livestock Scale Inspections New Mexico Pesticide Inspections Petroleum Standards Program Plant Inspections, Certifications and Quarantines Seed: General Information Standards & Consumer Services Apiaries New Mexico Chile Labeling Commercial Feed Registration Dairy Permits Egg Licensing Fertilizer/Soil Conditioner Registration Hemp Registration Nursery Licensing Pecan Buyer's Licenses & Registration Pesticide Compliance Weighmaster Licensing Weights & Measures Licensing & Registration News: Vineyard funding available to wine industry Enroll to be a listeria control specialist Cheeseburgers and salsas battle it out at fair Products of enchantment at state fair NM Food Protection Alliance Conference held Aug. 23 Pecan weevil quarantine hearings set for Aug. and Sept. Post-fire ag workshop in Mora Aug. 16 More NMDA News Resources Events: AG Day AgriFuture Educational Institute Rounders Award Ceremony Jeff M. Witte, Director/Secretary of Agriculture Jeff grew up on his family’s ranch on the Rowe Mesa between Moriarty and Las Vegas, New Mexico. He graduated from New Mexico State University (NMSU) with a bachelor’s degree in agricultural business management and a master’s degree in agricultural economics and economics. After college, Jeff worked for New Mexico Farm & Livestock Bureau as the field representative for the counties in northern New Mexico. He also represented the state’s farmers and ranchers in legislative discussions in Santa Fe. During this time, Jeff worked to bring farmer groups and rancher groups together. The Ag Group, as it’s loosely known, still gathers every year before the legislative session to set policy priorities that will benefit all of agriculture. When the session gets going, the group hosts Ag Fest to showcase to legislators and their staff the diversity of New Mexico agriculture. Jeff was instrumental in making this an annual event. Jeff first came to the New Mexico Department of Agriculture (NMDA), located in Las Cruces, in 1994. He spent the next nine years as the assistant director alongside Director/Secretary Frank DuBois. During that time, Jeff helped create the New Mexico Agricultural Leadership Program, a two-year program that aims to develop effective leaders within the state’s food, agriculture and natural resource sectors. Jeff then went to work as the director for the Office of Agricultural Biosecurity for NMDA and NMSU. He later started up the Southwest Border Food Safety and Defense Center at NMSU. In both capacities, he worked with private industry and officials at all levels of government to develop plans that will protect New Mexico agriculture as part of a homeland security strategy. Jeff still heads up trainings on the subject from time to time. Jeff was named New Mexico’s fifth permanent Director/Secretary of Agriculture in May 2011. He is past president of the National Association of State Departments of Agriculture (NASDA), past president of the Western Association of State Departments of Agriculture, past president of the Western United States Agriculture Trade Association, and past chair of Natural Resources, Pesticide Management & Environment Committee for NASDA. Jeff was appointed to the Local Government Advisory Committee (LGAC) by the U.S. Environmental Protection Agency Administrator in 2015, served as vice chairman from May 2018 to May 2021 and currently chairs the Healthy Communities work group. As a member of the LGAC, Jeff represents state-appointed officials. The committee provides advice and recommendation to the EPA Administrator to assist in developing a stronger partnership with local governments. The committee also provides direction regarding building state and local capacity to deliver environmental services and programs. In 2020, Jeff was appointed by the U.S. Environmental Protection Agency as a new member of the Farm, Ranch, and Rural Communities Committee (FRRCC). The FRRCC provides independent policy advice, information and recommendations to the EPA regarding environmental issues and policies that are of importance to agriculture and rural communities. His term ends June 2023. Jeff was also appointed by the U.S. Department of Agriculture as a new member of the Advisory Committee on Agriculture Statistics (ACAS) in 2020. The purpose of ACAS is to advise the U.S. Secretary of Agriculture and the National Agricultural Statics Service on the conduct of agricultural censuses and surveys. ACAS members represent a variety of agricultural sectors across several U.S. states and districts allowing for a broad representation of perspectives on various agricultural disciplines. He will serve as an ACAS member through June of 2022. In 2021, Jeff became a member of the NMSU Agriculture Modernization Advisory Committee (AMAC). NMSU’s College of Agricultural Consumer, and Environmental Sciences (ACES) launched a comprehensive initiative to update its buildings and programs and received funding under the 2018 and 2020 General Obligation Bonds. NMSU has ambitious plans to improve value-added agriculture, the health of New Mexicans and digitalization of agriculture in order to benefit farmers, ranchers and agricultural processors in the state. The main purpose of AMAC members is to advise, contribute expertise, make connections and serve as ambassadors for the overall mission and vision of updated agriculture facilities and programs. As a member of AMAC, Jeff will help contribute to the success of the overall project, make a positive impact on the agriculture community and higher education, and impact student learning as well as industry growth. NASDA President and New York Agriculture Commissioner Richard Ball appointed Jeff as vice chair of the association’s Natural Resources and Environment Committee in September 2021. The bulk of Jeff’s time as New Mexico’s secretary of agriculture is spent on the road, meeting with groups that represent farmers and ranchers across the state. He also works to educate legislators about New Mexico agriculture. Jeff and his wife Janet live in the southern New Mexico’s Mesilla Valley. Their son, Jeremy, received a degree in ag economics and ag business from NMSU, as well as a master’s degree in economics from George Mason University. He currently works for the U.S. Senate Committee on Agriculture as a senior staff member. Their daughter, Jennifer, received a bachelor’s degree in nursing from NMSU and currently works as a registered nurse in Albuquerque. Our website is not supported by any version of Internet Explorer. You can download a browser that is supported below. We apologize for any inconvenience. Download Chrome here Download Firefox here Main Office 3190 S. Espina Las Cruces, NM 88003 Phone: (575) 646-3007 Monday - Friday 8:00 a.m. - 5:00 p.m. Closed for lunch 12:00 p.m. - 1:00 p.m. Albuquerque District Office (ADO) 2604 Aztec NE Albuquerque, NM 87107 Phone: (505) 361-2840 Monday - Friday 8:00 a.m. - 5:00 p.m. Closed for lunch 12:00 p.m. - 1:00 p.m. NMDA Peanut Grading Station (PGS) 721 East Lime Street Portales, NM 88130 Phone: (575) 356-8393 Monday - Friday 8:00 a.m. - 5:00 p.m. Closed for lunch 12:00 p.m. - 1:00 p.m. Veterinary Diagnostic Services (VDS) 1101 Camino de Salud NE Albuquerque, NM 87102 (505) 383-9299 Phone (800) 432-9110 In State Only (505) 383-9294 Fax (505) 220-4489 After Hours Monday - Friday 7:30 a.m. - 4:30 p.m. Follow us Additional Information Directory Latest NM AG Stats Strategic Plan AG Reporting Hotline Employment Opportunities Wildlife Programs NMDA History Helpful Links Customer Concerns Accessibility © 2022 New Mexico Department of Agriculture. All Rights Reserved."

In [ ]:
doc = trained_nlp(texts)
displacy.render(doc, style="ent", jupyter=True)